In [75]:
import duckdb
import pandas as pd
from dotenv import dotenv_values
import datetime

from postgres_connect import PostgresConnection

In [2]:
config = dotenv_values()

In [3]:
pg = PostgresConnection(config=config)

In [6]:
game_details = pg.read_table('game_details')

/mnt/c/Users/jlpay/Documents/Dev/repos/Fact-Data-Modeling-Homework/postgres_connect.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, self._connection())


In [ ]:
duckdb.sql("""
           with dupes as (
           select
            *,
            row_number() over (partition by game_id, player_id) rn
           from game_details)
           
           select 
            * 
           from dupes 
           where 
            rn = 1""")
    

┌──────────┬────────────┬───────────────────┬─────────────┬───────────┬─────────────────────┬──────────┬────────────────┬──────────────────────────────────────────┬─────────┬────────┬────────┬────────┬────────┬────────┬─────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────────┬───────┐
│ game_id  │  team_id   │ team_abbreviation │  team_city  │ player_id │     player_name     │ nickname │ start_position │                 comment                  │   min   │  fgm   │  fga   │ fg_pct │  fg3m  │  fg3a  │ fg3_pct │  ftm   │  fta   │ ft_pct │  oreb  │  dreb  │  reb   │  ast   │  stl   │  blk   │   TO   │   pf   │  pts   │ plus_minus │  rn   │
│  int64   │   int64    │      varchar      │   varchar   │   int64   │       varchar       │ varchar  │    varchar     │                 varchar                  │ varchar │ double │ double │ double │ double │ double │ double  │ double │ double │ double │ double │ double │ double 

In [14]:
duckdb.sql("""
select
    distinct *
from game_details
where
    game_id = 22000043   
    and player_id = 201144 
""")

┌──────────┬────────────┬───────────────────┬───────────┬───────────┬─────────────┬──────────┬────────────────┬─────────┬─────────┬────────┬────────┬────────┬────────┬────────┬─────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────┬────────────┐
│ game_id  │  team_id   │ team_abbreviation │ team_city │ player_id │ player_name │ nickname │ start_position │ comment │   min   │  fgm   │  fga   │ fg_pct │  fg3m  │  fg3a  │ fg3_pct │  ftm   │  fta   │ ft_pct │  oreb  │  dreb  │  reb   │  ast   │  stl   │  blk   │   TO   │   pf   │  pts   │ plus_minus │
│  int64   │   int64    │      varchar      │  varchar  │   int64   │   varchar   │ varchar  │    varchar     │ varchar │ varchar │ double │ double │ double │ double │ double │ double  │ double │ double │ double │ double │ double │ double │ double │ double │ double │ double │ double │ double │   double   │
├──────────┼────────────┼───────────────────┼───────────┼───────────┼───────

In [70]:
df = pg.read_table('devices')

In [45]:
df.head()

,url,referrer,user_id,device_id,host,event_time
0,/,None,NaN,9.906757e+18,www.zachwilson.tech,2023-01-09 11:55:28.032000
1,/,None,1.664713e+19,9.906757e+18,www.zachwilson.tech,2023-01-09 20:10:27.610000
2,/,None,NaN,1.204839e+19,admin.zachwilson.tech,2023-01-10 04:43:49.204000
3,/,None,NaN,1.204839e+19,www.zachwilson.tech,2023-01-14 05:25:03.497000
4,/,None,NaN,1.204839e+19,www.zachwilson.tech,2023-01-14 05:25:04.305000


In [71]:
with duckdb.connect('events.db') as con:
    con.sql("DROP TABLE if EXISTS devices")
    con.sql("CREATE TABLE devices (device_id text, browser_type text, device_type text, os_type text)")
    con.sql("""
insert into devices
select device_id, browser_type, device_type, os_type from df
""")

In [68]:
with duckdb.connect('events.db') as con:
    con.sql("DROP TABLE IF EXISTS events;")
    con.sql("CREATE TABLE events (url text, referrer text, user_id text, device_id text, host text, event_time timestamp)")
    con.sql("""
insert into events
select * from df
""")

In [33]:
with duckdb.connect('events.db') as con:
    con.sql("select min(event_time), max(event_time) from events").show()

┌─────────────────────────┬─────────────────────────┐
│     min(event_time)     │     max(event_time)     │
│        timestamp        │        timestamp        │
├─────────────────────────┼─────────────────────────┤
│ 2023-01-01 00:06:50.079 │ 2023-01-31 23:51:51.685 │
└─────────────────────────┴─────────────────────────┘



In [31]:
with duckdb.connect('events.db') as con:
    con.sql("select * from events").show()

┌─────────────────────────────────────────┬────────────────────────────────┬───────────────┬───────────────┬───────────────────────┬─────────────────────────┐
│                   url                   │            referrer            │    user_id    │   evice_id    │         host          │       event_time        │
│                 varchar                 │            varchar             │     float     │     float     │        varchar        │        timestamp        │
├─────────────────────────────────────────┼────────────────────────────────┼───────────────┼───────────────┼───────────────────────┼─────────────────────────┤
│ /                                       │ NULL                           │          NULL │  9.906757e+18 │ www.zachwilson.tech   │ 2023-01-09 11:55:28.032 │
│ /                                       │ NULL                           │  1.664713e+19 │  9.906757e+18 │ www.zachwilson.tech   │ 2023-01-09 20:10:27.61  │
│ /                                       │ NU

In [51]:
with duckdb.connect('events.db') as con:
    con.sql("""select 
                * 
            from events e
            join devices d
                on e.device_id = e.device_id
            where
                e.user_id is not null""").show()

┌──────────────┬──────────┬───────────────┬───────────────┬─────────────────────┬─────────────────────────┬───────────────┬────────────────┬─────────────┬──────────┐
│     url      │ referrer │    user_id    │   device_id   │        host         │       event_time        │   device_id   │  browser_type  │ device_type │ os_type  │
│   varchar    │ varchar  │     float     │     float     │       varchar       │        timestamp        │     float     │    varchar     │   varchar   │ varchar  │
├──────────────┼──────────┼───────────────┼───────────────┼─────────────────────┼─────────────────────────┼───────────────┼────────────────┼─────────────┼──────────┤
│ /            │ NULL     │  1.664713e+19 │  9.906757e+18 │ www.zachwilson.tech │ 2023-01-09 20:10:27.61  │  8.971692e+18 │ Chrome         │ Other       │ Mac OS X │
│ /            │ NULL     │  1.664713e+19 │  9.906757e+18 │ www.zachwilson.tech │ 2023-01-09 20:10:27.61  │ 2.5693874e+18 │ FacebookBot    │ Spider      │ Other    │
│ / 

In [60]:
with duckdb.connect('events.db') as con:
    con.sql("""
            with daily as (
            select 
                e.user_id,
                d.device_id,
                d.browser_type,
                count(*) total
            from events e
            join devices d
                on e.device_id = e.device_id
            where
                e.user_id is not null
                and e.event_time::Date  = '2023/1/1'
            group by all)
            
            select * from daily
            where total = 1""").show()

┌─────────┬───────────┬──────────────┬───────┐
│ user_id │ device_id │ browser_type │ total │
│  float  │   float   │   varchar    │ int64 │
├─────────┴───────────┴──────────────┴───────┤
│                   0 rows                   │
└────────────────────────────────────────────┘



In [ ]:
with duckdb.connect('events.db') as con:
    # con.sql("drop table if exists user_devices_cumulated")
    # con.sql("create table user_devices_cumulated (userid text, browser_dates map(text, date))")
    con.sql("""
with today_dupe as (
select
    e.user_id,
    d.device_id,
    d.browser_type,
    row_number() over (partition by e.user_id, d.device_id) rn
from events e
    left join devices d 
        on d.device_id = e.device_id
where
    user_id is not null
            and event_time::date = '2023/1/1/'),
today as (
select
    *
from today_dupe
where
    rn = 1), mapped_cols as (
select
    user_id,
    MAP { browser_type: 1 } as browser_count
from today
where
    browser_type is not null),
joined as (
            
select
    mc1.user_id,
    mc1.browser_count || mc2.browser_count as browser_count,
    mc1.browser_count,
    mc2.browser_count,
    /*case
        when mc2.browser_count is not null then mc1.browser_count := mc2.browser_count
    end browser_count*/
from mapped_cols mc1
    join mapped_cols mc2
        on mc1.user_id = mc2.user_id
        and mc1.browser_count <> mc2.browser_count
        and mc1.browser_count is not null)
            
select * from joined
 """).show()

InvalidInputException: Invalid Input Error: Invalid map argument(s). Valid map arguments are a list of key-value pairs (MAP {'key1': 'val1', ...}), two lists (MAP ([1, 2], [10, 11])), or no arguments.

In [122]:
with duckdb.connect('events.db') as con:
    con.sql("drop table if exists user_devices_cumulated")
    con.sql("create table user_devices_cumulated (user_id text, browser_type text, dates date[], current_date date)")
    con.sql("""
with today_dupe as (
select
    e.user_id,
    d.device_id,
    d.browser_type,
    row_number() over (partition by e.user_id, d.device_id) rn
from events e
    left join devices d 
        on d.device_id = e.device_id
where
    user_id is not null
            and event_time::date = '2023/1/1/'),
today as (
select
    *
from today_dupe
where
    rn = 1), mapped_cols as (
select
    user_id,
    browser_type,
    ['2023/1/1'::date] as date,
    '2023/1/1'::date current_date
from today
where
    browser_type is not null),
yesterday as (
select
    *
from user_devices_cumulated
where
    current_date = current_date
            ),
historical as (
select
    *
from yesterday y
where
    not exists (select * from mapped_cols mc where mc.user_id = y.user_id and mc.browser_type = y.browser_type)),
new as (
select
    *
from mapped_cols mc
where
    not exists (select * from yesterday y where mc.user_id = y.user_id and mc.browser_type = y.browser_type)),
updated as (
select
    user_id,
    browser_type,
    dates || ['2023/1/1'::Date],
    '2023/1/1'::Date current_date
from yesterday y
where
    exists (select * from mapped_cols mc where mc.user_id = y.user_id and mc.browser_type = y.browser_type))
            
                   
select * from historical
union all
select * from new
union all
select * from updated
 """).show()

┌────────────────────────┬───────────────┬──────────────┬──────────────┐
│        user_id         │ browser_type  │    dates     │ current_date │
│        varchar         │    varchar    │    date[]    │     date     │
├────────────────────────┼───────────────┼──────────────┼──────────────┤
│ 2.149685481453258e+18  │ ) Bot         │ [2023-01-01] │ 2023-01-01   │
│ 1.36472210049767e+19   │ AhrefsBot     │ [2023-01-01] │ 2023-01-01   │
│ 2.0144825980926515e+18 │ AhrefsBot     │ [2023-01-01] │ 2023-01-01   │
│ 2.794586222856017e+18  │ SemrushBot    │ [2023-01-01] │ 2023-01-01   │
│ 1.53423419884343e+19   │ Googlebot     │ [2023-01-01] │ 2023-01-01   │
│ 1.1834646844374e+19    │ Chrome        │ [2023-01-01] │ 2023-01-01   │
│ 5.017882618361265e+18  │ Chrome        │ [2023-01-01] │ 2023-01-01   │
│ 1.33940240807639e+19   │ Chrome        │ [2023-01-01] │ 2023-01-01   │
│ 1.36508603897267e+19   │ Googlebot     │ [2023-01-01] │ 2023-01-01   │
│ 1.4322833487723359e+18 │ bingbot       │ [2023-01

In [132]:
int('2024/11/23'.split('/')[0])

2024

In [136]:
def load_browser_dates(date: datetime.date) -> None:    
    if isinstance(date, str):
        date_list = date.split('/')
        date = datetime.date(int(date_list[0]), int(date_list[1]), int(date_list[2]))
    with duckdb.connect('events.db') as con:
        con.sql(f"""
    with today_dupe as (
    select
        e.user_id,
        d.device_id,
        d.browser_type,
        row_number() over (partition by e.user_id, d.device_id) rn
    from events e
        left join devices d 
            on d.device_id = e.device_id
    where
        user_id is not null
                and event_time::date = {date}),
    today as (
    select
        *
    from today_dupe
    where
        rn = 1), mapped_cols as (
    select
        user_id,
        browser_type,
        ['{date}'::date] as date,
        '{date}'::date current_date
    from today
    where
        browser_type is not null),
    yesterday as (
    select
        *
    from user_devices_cumulated
    where
        current_date = '{date} interval -1'
                ),
    historical as (
    select
        *
    from yesterday y
    where
        not exists (select * from mapped_cols mc where mc.user_id = y.user_id and mc.browser_type = y.browser_type)),
    new as (
    select
        *
    from mapped_cols mc
    where
        not exists (select * from yesterday y where mc.user_id = y.user_id and mc.browser_type = y.browser_type)),
    updated as (
    select
        user_id,
        browser_type,
        dates || ['{date}'::Date],
        '{date}'::Date current_date
    from yesterday y
    where
        exists (select * from mapped_cols mc where mc.user_id = y.user_id and mc.browser_type = y.browser_type))
                
                    
    select * from historical
    union all
    select * from new
    union all
    select * from updated
    """)
        
load_browser_dates('2023/1/2')

In [230]:
with duckdb.connect('events.db') as con:
    con.sql("drop table if exists date_list_int")
    con.sql("create table date_list_int (host text, date_list_int integer[])")

In [231]:
def load_date_list(date: datetime.date) -> None:
    if isinstance(date, str):
        date_split = date.split('/')
        date = datetime.date(int(date_split[0]), int(date_split[1]), int(date_split[2]))
    first_month = date.replace(day=1)
    with duckdb.connect('events.db') as con:
        con.sql(f"""
                insert into date_list_int
                with today as (
                select 
                    distinct host
                from events
                where
                    event_time::date = '{date}')
                
                select
                    t.host,
                    case
                        when t.host is null and dli.host is not null then dli.date_list_int || [1]
                        when dli.host is null then ARRAY({[0 for i in range(abs((date -first_month).days))]}) || [1]
                        when t.host is not null then dli.date_list_int || [1]
                        else [1]
                    end as date_list_int
                from today t
                    full outer join date_list_int dli
                        on t.host = dli.host
                """)
        
load_date_list('2023/1/1')

ParserException: Parser Error: syntax error at or near "["

In [232]:
with duckdb.connect('events.db') as con:
    con.sql("""
            select 
                * 
            from date_list_int
            """).show()

┌─────────┬───────────────┐
│  host   │ date_list_int │
│ varchar │    int32[]    │
├─────────┴───────────────┤
│         0 rows          │
└─────────────────────────┘



'[0, 0, 0]'